In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 10)

pd.set_option('display.max_columns', 15)


read_loan_with_8C = pd.read_csv("Files/loan_approval_8_columns.csv")

read_approval_dataSet_extra = pd.read_csv("Files/loan_approval_dataset_extra.csv")

# print(read_approval_dataSet_extra.columns)

df_extra_selected = read_approval_dataSet_extra[[' no_of_dependents', ' education',
                                                 ' self_employed', ' income_annum', ' loan_amount', ' commercial_assets_value',
                                                 ' luxury_assets_value', ' bank_asset_value']]

# print(df_extra_selected)

combined_loan_dataset = pd.concat([read_loan_with_8C, df_extra_selected], axis=1)

print(combined_loan_dataset)

combined_loan_dataset.to_csv("Files/loan_approval_preprocessed.csv", index= False)


                 name              city    income  credit_score  loan_amount  \
0        Allison Hill         East Jill  113810.0         389.0      39698.0   
1        Brandon Hall     New Jamesside   44592.0         729.0      15446.0   
2        Rhonda Smith      Lake Roberto   33278.0         584.0      11189.0   
3     Gabrielle Davis  West Melanieview  127196.0         344.0      48823.0   
4        Valerie Gray         Mariastad   66048.0         496.0      47174.0   
...               ...               ...       ...           ...          ...   
4264              NaN               NaN       NaN           NaN          NaN   
4265              NaN               NaN       NaN           NaN          NaN   
4266              NaN               NaN       NaN           NaN          NaN   
4267              NaN               NaN       NaN           NaN          NaN   
4268              NaN               NaN       NaN           NaN          NaN   

      years_employed  points  ...      

In [3]:
from numpy import exceptions
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)

# read from load data loan approval preprocessed

try:
  loan_approval_preprocessed = pd.read_csv("Files/loan_approval_preprocessed.csv")
  print(loan_approval_preprocessed.tail(10))

except Exception as ex:
  print(f"error: {ex}")


#imputation
# Calculate monthly income from 'income_annum' and assign it to the 'income' column.
# This is done using vectorized operations for efficiency.
if ' income_annum' in loan_approval_preprocessed.columns:
    loan_approval_preprocessed['income'] = (loan_approval_preprocessed[' income_annum'] / 12).round(2)

else:
    print("Warning: 'income_annum' column not found. Cannot calculate monthly income.")

# Define a robust imputation function that can handle both numerical/categorical and boolean columns
def sample_data_imputation(columns, is_boolean=False):
  missing_mask = loan_approval_preprocessed[columns].isna()
  n_missing = missing_mask.sum()

  if n_missing == 0:
    return # No missing values, nothing to do

  existing_values = loan_approval_preprocessed[columns].dropna()

  if is_boolean:
    # Check if existing_values are actually binary before calculating mean for binomial distribution
    if existing_values.isin([0, 1]).all():
      p_success = existing_values.mean()
      samples_for_imputation = np.random.binomial(n=1, p=p_success, size=n_missing)
    else:
      print(f"Warning: Column '{columns}' is marked as boolean but contains non-binary values. Falling back to random sampling.")
      samples_for_imputation = np.random.choice(existing_values, size=n_missing, replace=True)
  else:
    samples_for_imputation = np.random.choice(existing_values, size=n_missing, replace=True)

  loan_approval_preprocessed.loc[missing_mask, columns] = samples_for_imputation


#filling in the credit score column
sample_data_imputation(columns='credit_score')

#filling in the loan amount column
sample_data_imputation(columns='loan_amount')

#filling in the years of employment column
sample_data_imputation(columns='years_employed')

#filling in the points column
sample_data_imputation(columns='points')

#filling in the loan_approved column (boolean)
sample_data_imputation(columns='loan_approved', is_boolean=True)

#filling in the name column
sample_data_imputation(columns='name')

#filling in the city column
sample_data_imputation(columns='city')


loan_approval_preprocessed.to_csv("Files/loan_approval_preprocessed.csv", index=False)

                       name               city     income  credit_score  \
4259         Courtney Smith     East Craigfurt  375000.00         785.0   
4260           Meagan Smith         Port Chloe  733333.33         744.0   
4261  Mr. William Davis DDS      Dennischester  250000.00         333.0   
4262             Chad Reyes      North Jonfurt  108333.33         717.0   
4263           Tonya Oliver     New Kelseyview  416666.67         325.0   
4264           Linda Miller         New Kristy   83333.33         464.0   
4265            Lori Brooks     Kristenchester  275000.00         645.0   
4266        Elizabeth Clark          Lake Ryan  541666.67         362.0   
4267      Victoria Williams  Port Kathyborough  341666.67         804.0   
4268         Christina Cobb        West Walter  766666.67         724.0   

      loan_amount  years_employed  points  ...      education   self_employed  \
4259       9183.0            33.0    35.0  ...   Not Graduate             Yes   
4260      25